In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
cd drive/MyDrive/soccerMamba_project/'AI demo in video'/

/content/drive/MyDrive/soccerMamba_project/AI demo in video


In [ ]:
ls

 AdobeStock_531668498_Video_HD_Preview.mp4   ligapro_demo_4_output.mp4
 Demo_using_SAM.ipynb                        ligapro_demo_5.ipynb
 ligapro_2025_2.mp4                          ligapro_demo_6.ipynb
'ligapro_2025_2_output (1).mp4'              ligapro_demo.ipynb
 ligapro_2025_3_output.mp4                   output_soccer.mp4
 ligapro_2025.mp4                            sam_vit_b_01ec64.pth
 ligapro_2025_output.mp4                     soccer_demo.ipynb
 ligapro_demo_2.ipynb                        videoplayback_complete.mp4
 ligapro_demo_3.ipynb                        videos/
 ligapro_demo_4.ipynb                        yolov8n.pt


In [ ]:
video_path = 'videos/ligapro_2025_2.mp4'

In [ ]:
import cv2

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
else:
    print("Video opened successfully!")


Video opened successfully!


In [ ]:
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install opencv-python ultralytics


  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-gxhgo0lm
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-gxhgo0lm
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36592 sha256=989e18e197b5c0a1a567ba1cdb86596e6204c4223dbb1cd0ee105708ebb394f2
  Stored in directory: /tmp/pip-ephem-wheel-cache-197ql6e_/wheels/15/d7/bd/05f5f23b7dcbe70cbc6783b06f12143b0cf1a5da5c7b52dcc5
Successfully built segment_anything
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import cv2
import torch
import numpy as np
from segment_anything import sam_model_registry, SamPredictor
from ultralytics import YOLO
from tqdm import tqdm


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Your paths in Google Drive
input_video_path = 'videos/ligapro_2025_2.mp4'
output_video_path = 'videos/ligapro_2025_SAM.mp4'


In [ ]:
# YOLOv8 for person detection
yolo_model = YOLO('yolov8n.pt')

# Download SAM checkpoint if you don't have it yet
!wget -nc https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth

# Load SAM
device = "cuda" if torch.cuda.is_available() else "cpu"
sam_checkpoint = "sam_vit_b_01ec64.pth"
sam = sam_model_registry["vit_b"](checkpoint=sam_checkpoint)
sam.to(device)
predictor = SamPredictor(sam)



File ‘sam_vit_b_01ec64.pth’ already there; not retrieving.



In [ ]:
# Open input video
cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Output video writer (grayscale mask video)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height), isColor=True)

print(f"Input FPS: {fps}, Size: {width}x{height}, Total frames: {total_frames}")


# New writer for COLOR video

for _ in tqdm(range(total_frames)):
    ret, frame = cap.read()
    if not ret:
        break

    # 1️⃣ Detect players
    results = yolo_model(frame)
    boxes = []
    for det in results[0].boxes.xyxy.cpu().numpy():
        boxes.append(det)

    if not boxes:
        # No detections → just write the original frame
        out.write(frame)
        continue

    # 2️⃣ SAM prediction
    predictor.set_image(frame)
    transformed_boxes = predictor.transform.apply_boxes_torch(
        torch.tensor(boxes, device=device),
        frame.shape[:2]
    )

    masks, _, _ = predictor.predict_torch(
        point_coords=None,
        point_labels=None,
        boxes=transformed_boxes,
        multimask_output=False
    )

    # 3️⃣ Combine all masks into single binary mask
    #combined_mask = torch.any(masks, dim=0).cpu().numpy().astype(np.uint8) * 255
    combined_mask = torch.any(masks, dim=0).cpu().numpy().astype(np.uint8)
    combined_mask = combined_mask.squeeze() * 255

    # 4️⃣ Make a colored overlay
    overlay_color = (0, 0, 255)  # Red in BGR
    mask_color = np.zeros_like(frame)
    mask_color[:, :] = overlay_color

    # 5️⃣ Apply alpha blending
    alpha = 0.4
    mask_bool = combined_mask.astype(bool)

    blended_frame = np.where(
        mask_bool[:, :, None],
        (frame * (1 - alpha) + mask_color * alpha).astype(np.uint8),
        frame
    )


    # 6️⃣ Write blended frame
    out.write(blended_frame)

cap.release()
out.release()

Input FPS: 30.0488984699511, Size: 854x480, Total frames: 381


  0%|          | 0/381 [00:00<?, ?it/s]


0: 384x640 15 persons, 21.3ms
Speed: 4.4ms preprocess, 21.3ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)


  0%|          | 1/381 [00:00<03:41,  1.71it/s]


0: 384x640 13 persons, 12.9ms
Speed: 3.0ms preprocess, 12.9ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)


...

 99%|█████████▉| 379/381 [02:53<00:00,  2.23it/s]


0: 384x640 2 persons, 10.8ms
Speed: 2.7ms preprocess, 10.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)


100%|█████████▉| 380/381 [02:54<00:00,  2.24it/s]


0: 384x640 2 persons, 18.0ms
Speed: 2.7ms preprocess, 18.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)


100%|██████████| 381/381 [02:54<00:00,  2.18it/s]
